---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
len(states)

57

## Question 1

In [3]:
UT=pd.read_table('university_towns.txt', header = None)
UT.head(10)

,0
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]
5,Montevallo (University of Montevallo)[2]
6,Troy (Troy University)[2]
7,"Tuscaloosa (University of Alabama, Stillman Co..."
8,Tuskegee (Tuskegee University)[5]
9,Alaska[edit]


In [4]:
UT_State = UT.copy()
UT_State = UT_State[UT_State[0].str.contains(r"\[edit\]")]


Type  <class 'pandas.core.frame.DataFrame'>
Length  50


In [5]:
UT_State_List = UT_State[0].str.replace(r"\[.*\]","").str.strip().tolist()
len(UT_State_List)

50

In [7]:
UT[0]=UT[0].str.replace(r"\(.*\)","").str.replace(r"\[.*\]","").str.strip()
print("Type ",type(UT))
print("Length ",len(UT))
UT.head(10)

Type  <class 'pandas.core.frame.DataFrame'>
Length  567


,0
0,Alabama
1,Auburn
2,Florence
3,Jacksonville
4,Livingston
5,Montevallo
6,Troy
7,Tuscaloosa
8,Tuskegee
9,Alaska


In [8]:
UT.loc[0,0]

'Alabama'

In [9]:
UT.loc[0,0] in UT_State_List

True

In [10]:
data = []

for index, row in UT.iterrows():
    if(UT.loc[index,0] in UT_State_List):
        state = UT.loc[index,0]
        UT_State_List.remove(state)
    else:
        town = UT.loc[index,0]
        data.append([state,town])

In [11]:
unitowns = pd.DataFrame(data,columns=["State","RegionName"])
unitowns.shape

(517, 2)

In [12]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    UT=pd.read_table('university_towns.txt', header = None)
    
    UT_State = UT[UT[0].str.contains(r"\[edit\]")]
    UT_State_List = UT_State[0].str.replace(r"\[.*\]","").str.strip().tolist()
    
    UT[0]=UT[0].str.replace(r"\(.*","").str.replace(r"\[.*\]","").str.rstrip()
    
    data = []

    for index, row in UT.iterrows():
        if(UT.loc[index,0] in UT_State_List):
            state = UT.loc[index,0]
            UT_State_List.remove(state)
        else:
            town = UT.loc[index,0]
            data.append([state,town])

    unitowns = pd.DataFrame(data,columns=["State","RegionName"])
    
    return unitowns

In [15]:
df = get_list_of_university_towns()

In [16]:
cols = ["State", "RegionName"]

print('Shape test: ', "Passed" if df.shape ==
      (517, 2) else 'Failed')

Shape test:  Passed


In [17]:
print('Index test: ',
      "Passed" if df.index.tolist() == list(range(517))
      else 'Failed')

Index test:  Passed


In [18]:
print('Column test: ',
      "Passed" if df.columns.tolist() == cols else 'Failed')

Column test:  Passed


In [19]:
print('\\n test: ',
      "Failed" if any(df[cols[0]].str.contains(
          '\n')) or any(df[cols[1]].str.contains('\n'))
      else 'Passed')

\n test:  Passed


In [20]:
print('Trailing whitespace test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\s+$')) or any(df[cols[1]].str.contains(
              '\s+$'))
      else 'Passed')

Trailing whitespace test: Passed


In [21]:
print('"(" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\(')) or any(df[cols[1]].str.contains(
              '\('))
      else 'Passed')

"(" test: Passed


## Question 2

In [68]:
GDP = pd.read_excel('gdplev.xls',skiprows=220,header = None)
GDP.head(10)

,0,1,2,3,4,5,6,7
0,NaN,NaN,NaN,NaN,2000q1,10031.0,12359.1,NaN
1,NaN,NaN,NaN,NaN,2000q2,10278.3,12592.5,NaN
2,NaN,NaN,NaN,NaN,2000q3,10357.4,12607.7,NaN
3,NaN,NaN,NaN,NaN,2000q4,10472.3,12679.3,NaN
4,NaN,NaN,NaN,NaN,2001q1,10508.1,12643.3,NaN
5,NaN,NaN,NaN,NaN,2001q2,10638.4,12710.3,NaN
6,NaN,NaN,NaN,NaN,2001q3,10639.5,12670.1,NaN
7,NaN,NaN,NaN,NaN,2001q4,10701.3,12705.3,NaN
8,NaN,NaN,NaN,NaN,2002q1,10834.4,12822.3,NaN
9,NaN,NaN,NaN,NaN,2002q2,10934.8,12893.0,NaN


In [70]:
GDP=GDP[[4,5,6]]
print(GDP.shape)
GDP.head(10)

(66, 3)


,4,5,6
0,2000q1,10031.0,12359.1
1,2000q2,10278.3,12592.5
2,2000q3,10357.4,12607.7
3,2000q4,10472.3,12679.3
4,2001q1,10508.1,12643.3
5,2001q2,10638.4,12710.3
6,2001q3,10639.5,12670.1
7,2001q4,10701.3,12705.3
8,2002q1,10834.4,12822.3
9,2002q2,10934.8,12893.0


In [73]:
recession_start = []
for i in range(len(GDP) - 2):
    if(GDP.iloc[i][5] > GDP.iloc[i+1][5]) & (GDP.iloc[i+1][5] > GDP.iloc[i+2][5]):
        recession_start.append(GDP.iloc[i][4])

In [74]:
recession_start

['2008q3', '2008q4']

In [77]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls',skiprows=220,header = None)
    GDP=GDP[[4,5,6]]
    
    recession_start = []
    for i in range(len(GDP) - 2):
        if(GDP.iloc[i][5] > GDP.iloc[i+1][5]) & (GDP.iloc[i+1][5] > GDP.iloc[i+2][5]):
            recession_start.append(GDP.iloc[i][4])
    
    return recession_start[0]

In [92]:
recession_start = get_recession_start()
recession_start

'2008q3'

## Problem 3

In [89]:
recession_end = []
for i in range(len(GDP) - 2):
    if(GDP.iloc[i][5] < GDP.iloc[i+1][5]) & (GDP.iloc[i+1][5] < GDP.iloc[i+2][5]):
        recession_end.append(GDP.iloc[i+2][4])

In [93]:
recession_end = [i for i in recession_end if i > recession_start]

In [94]:
recession_end[0]

'2009q4'

In [95]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    recession_end = []
    for i in range(len(GDP) - 2):
        if(GDP.iloc[i][5] < GDP.iloc[i+1][5]) & (GDP.iloc[i+1][5] < GDP.iloc[i+2][5]):
            recession_end.append(GDP.iloc[i+2][4])
    
    recession_end = [i for i in recession_end if i > recession_start]
    
    return recession_end[0]

In [97]:
recession_end = get_recession_end()
recession_end

'2009q4'

## Problem 4

### Approach 1

In [125]:
GDP[GDP[4] == recession_start].index[0]

34

In [126]:
GDP[GDP[4] == recession_end].index[0]

39

In [127]:
GDP.iloc[34:39]

,4,5,6
34,2008q3,14843.0,14891.6
35,2008q4,14549.9,14577.0
36,2009q1,14383.9,14375.0
37,2009q2,14340.4,14355.6
38,2009q3,14384.1,14402.5


In [128]:
recession = GDP.iloc[GDP[GDP[4] == recession_start].index[0]:GDP[GDP[4] == recession_end].index[0]]
recession[recession[5]==recession[5].min()].iloc[0][4]

'2009q2'

### Approach 2

In [112]:
recession = GDP[(GDP[4] > recession_start) & (GDP[4] < recession_end)]
recession

,4,5,6
35,2008q4,14549.9,14577.0
36,2009q1,14383.9,14375.0
37,2009q2,14340.4,14355.6
38,2009q3,14384.1,14402.5


In [119]:
recession[recession[5]==recession[5].min()].iloc[0][4]

'2009q2'

In [120]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    recession = GDP[(GDP[4] > recession_start) & (GDP[4] < recession_end)]
    
    return recession[recession[5]==recession[5].min()].iloc[0][4]

In [121]:
get_recession_bottom()

'2009q2'

## Problem 5

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"